# **Script para monitorear cambios en sitios web**

In [ ]:
import requests
import hashlib
import os
from google.colab import drive
import re
import ssl
from bs4 import BeautifulSoup, Comment # Importar BeautifulSoup y Comment

# --- Configuración ---
# Lista de URLs a monitorear
urls_a_monitorear = [
    "https://buenosaires.gob.ar/jefaturadegabinete/desarrollo-urbano/normativa/codigo-urbanistico-y-de-edificacion",
    "https://buenosaires.gob.ar/jefaturadegabinete/desarrollo-urbano/novedades-de-la-subsecretaria-de-gestion-urbana",
    "https://buenosaires.gob.ar/agc/noticias",
    "https://buenosaires.gob.ar/noticias/desarrollo-urbano",
    "https://adrianmercadorealestate.com/blog/informes",
    "https://www.estadisticaciudad.gob.ar/eyc/?page_id=1479",
    "https://www.ceso.com.ar/secciones/provincias-y-regiones",
    "https://colegioinmobiliario.org.ar/institucional/observatorioEstadistico",
    "https://www.colliers.com/es-ar",
    "https://www.afcp.org.ar/despacho-mensual",
    "https://www.indec.gob.ar/indec/web/Nivel3-Tema-3-3",
    "https://www.remax.com.ar/indice-remax",
    "https://www.ieric.org.ar/estadistica/informes-de-coyuntura/?2025",
    "https://www.uade.edu.ar/sites/investigacion/instituto-de-economia-ineco/informes-y-novedades/mei-informe-del-mercado-inmobiliario-e-indice-del-salario-real/",
    "https://www.nmrk.com.ar/informe/11023/reportes-de-mercado-2025",
    "https://www.fabianachaval.com/blog",
    "https://sites.google.com/view/red-operaciones-inmobiliarias",
    "https://www.ljramos.com.ar/informes-del-mercado-inmobiliario",
    "https://www.zonaprop.com.ar/noticias/zpindex/",
    "https://www.zonaprop.com.ar/noticias/zpindex/gba-venta/",
    "https://www.zonaprop.com.ar/noticias/zpindex/gba-oeste-sur-venta/",
    "https://colegioinmobiliario.org.ar/novedades",
    "https://www.cpau.org/noticias",
    "https://noticias.argenprop.com/"
]

# Directorio en Google Drive para guardar los hashes de los estados anteriores
# Asegúrate de que la carpeta 'web_monitoring_hashes' exista en la raíz de tu Drive
drive_base_path = '/content/drive/MyDrive/web_monitoring_hashes'

# --- Funciones ---

def get_page_content(url):
    """Obtiene el contenido HTML de una URL, manejando errores comunes."""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': url
    }
    try:
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()

        print(f"  Contenido obtenido exitosamente.")
        return response.text

    except requests.exceptions.HTTPError as e:
        print(f"  Error HTTP para {url}: {e}")
        if e.response.status_code == 403:
            print("    Posiblemente bloqueado por medidas anti-bot.")
        return None

    except requests.exceptions.SSLError as e:
        print(f"  Error SSL para {url}: {e}")
        print("    Puede haber un problema con el certificado del sitio o la verificación.")
        # --- INICIO: Código PELIGROSO (Desactivar verificación SSL) ---
        # Si ACEPTAS EL RIESGO para esta URL específica, puedes intentar descomentar:
        try:
            print("    Intentando sin verificar certificado SSL (NO SEGURO!)...")
            response = requests.get(url, headers=headers, timeout=15, verify=False)
            response.raise_for_status()
            print(f"    Contenido obtenido (sin verificación SSL).")
            return response.text
        except requests.exceptions.RequestException as inner_e:
            print(f"    Falló incluso sin verificación SSL: {inner_e}")
            return None
        # --- FIN: Código PELIGROSO ---
        return None

    except requests.exceptions.RequestException as e:
        print(f"  Otro error de conexión/solicitud para {url}: {e}")
        return None

    except Exception as e:
        print(f"  Ocurrió un error inesperado al procesar {url}: {e}")
        return None

def clean_html_for_hashing(html_content):
    """
    Limpia el contenido HTML para reducir la sensibilidad al ruido.
    Elimina scripts, styles, comentarios y algunos metadatos no esenciales.
    Normaliza espacios en blanco.
    """
    if not html_content:
        return ""

    try:
        soup = BeautifulSoup(html_content, 'html.parser')

        # Eliminar elementos de script y style
        for script_or_style in soup(["script", "style"]):
            script_or_style.decompose()

        # Eliminar comentarios
        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        # Opcional: Eliminar ciertos metadatos que cambian a menudo (ej. generador, viewport si no son importantes)
        # Cuidado con esto, algunos meta tags son importantes (ej. charset, description)
        for meta in soup.find_all('meta'):
            if meta.get('name') in ['generator', 'viewport']:
                meta.decompose()
        #     # Podrías añadir otras condiciones si identificas meta tags ruidosos

        # Opcional: Eliminar elementos específicos ruidosos (ej. contadores de visitas discretos)
        if soup.find('div', id='visitor-counter'):
            soup.find('div', id='visitor-counter').decompose()

        # Convertir el objeto BeautifulSoup limpio de vuelta a una cadena.
        # Esto preserva la estructura y los atributos de los elementos restantes.
        # El uso de formatter="html" y pretty_print=False ayuda a obtener una salida consistente
        # aunque puede variar ligeramente entre versiones de bs4 o parsers.
        # Usar .encode().decode() y lstrip() ayuda a normalizar la salida de bs4
        cleaned_html = str(soup.prettify(formatter=None)).strip()


        # Opcional: Si solo te interesa el texto visible (ignorar atributos, estructura)
        text_content = soup.get_text(separator=' ', strip=True)
        return text_content
        # PERO, hashing el HTML limpio es mejor para detectar cambios como alt text, href, etc.

        return cleaned_html

    except Exception as e:
        print(f"Error durante la limpieza del HTML: {e}")
        return "" # Retorna vacío si la limpieza falla

def calculate_hash(content):
    """Calcula el hash SHA256 del contenido."""
    if not content: # Manejar contenido vacío después de la limpieza o por error
        return "d41d8cd98f00b204e9800998ecf8427e" # Hash de una cadena vacía (MD5), o SHA256 de vacío: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
    return hashlib.sha256(content.encode('utf-8', errors='ignore')).hexdigest()

def get_filename_from_url(url):
    """Genera un nombre de archivo seguro basado en el hash de la URL."""
    url_hash = hashlib.md5(url.encode('utf-8')).hexdigest() # Usar MD5 para el nombre del archivo
    return f"{url_hash}.txt"

def save_last_hash(file_path, current_hash):
    """Guarda el hash actual en un archivo."""
    if not current_hash: # No guardar hash si el cálculo falló
         return
    try:
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w') as f:
            f.write(current_hash)
    except Exception as e:
        print(f"Error al guardar el hash en {file_path}: {e}")

def load_last_hash(file_path):
    """Carga el último hash guardado desde un archivo."""
    if os.path.exists(file_path):
        try:
            with open(file_path, 'r') as f:
                return f.read().strip()
        except Exception as e:
            print(f"Error al cargar el hash desde {file_path}: {e}")
            return None
    return None # No existe el archivo

# --- Ejecución principal ---

# 1. Montar Google Drive
print("Montando Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive montado.")
    drive_mounted = True
except Exception as e:
    print(f"Error al montar Google Drive: {e}")
    print("No se podrá guardar/cargar el estado anterior. Los cambios solo se detectarán en esta ejecución.")
    drive_mounted = False


# 2. Procesar cada URL
print(f"\nIniciando monitoreo de {len(urls_a_monitorear)} URL(s)...")
cambios_detectados = []
urls_con_error = []

for url in urls_a_monitorear:
    print(f"\n--- Procesando: {url} ---")
    current_content = get_page_content(url)

    if current_content is None:
        print(f"No se pudo obtener contenido de {url}. Saltando.")
        urls_con_error.append(url)
        continue

    # --- Limpiar contenido antes de hashear ---
    cleaned_content = clean_html_for_hashing(current_content)

    if not cleaned_content: # Si la limpieza resultó en contenido vacío (podría pasar si la página estaba vacía o la limpieza falló)
         print(f"Contenido limpio de {url} está vacío. No se puede monitorear.")
         urls_con_error.append(url) # Considerar como error si no hay contenido limpio
         continue

    current_hash = calculate_hash(cleaned_content)
    if current_hash is None: # Aunque calculate_hash maneja vacío, mantener por si acaso
        print(f"No se pudo calcular el hash de {url}. Saltando.")
        urls_con_error.append(url)
        continue

    if drive_mounted:
        url_state_filename = get_filename_from_url(url)
        url_state_path = os.path.join(drive_base_path, url_state_filename)

        last_hash = load_last_hash(url_state_path)

        # 3. Comparar y reportar
        if last_hash is None:
            print("  Estado anterior no encontrado (primera vez monitoreando o archivo eliminado). Guardando estado actual limpio.")
            save_last_hash(url_state_path, current_hash)
        elif current_hash != last_hash:
            print("  ¡CAMBIO DETECTADO!")
            cambios_detectados.append(url)
            save_last_hash(url_state_path, current_hash) # Actualizar el estado guardado
        else:
            print("  No se detectaron cambios.")
    else:
         print("  Google Drive no está montado. No se puede comparar con el estado anterior ni guardar el actual.")


# 4. Resumen de resultados
print("\n--- Resumen del Monitoreo ---")
if cambios_detectados:
    print(f"Se detectaron cambios en las siguientes {len(cambios_detectados)} URL(s):")
    for url_cambiada in cambios_detectados:
        print(f"- {url_cambiada}")
else:
    print("No se detectaron cambios en las URLs procesadas (o era la primera ejecución para algunas, o hubo errores).")

if urls_con_error:
    print(f"\nOcurrieron errores o problemas de contenido al procesar las siguientes {len(urls_con_error)} URL(s):")
    for url_error in urls_con_error:
        print(f"- {url_error}")
    print("\nRevisa los mensajes de error detallados arriba para cada una de estas URLs.")

# Desmontar Drive al finalizar (opcional)
# print("\nDesmontando Google Drive...")
# drive.flush_and_unmount()
# print("Google Drive desmontado.")